In [1]:
# Import DS tools
import pandas as pd

# Import project specific functions
from scripts import data

# Import data
data_dict = data.get_clean_data_dict()
purchases = data_dict['purchases']
users = data_dict['users']

### Target data-frame analysis and munging
Our goal is to predict the value of the purchases each `user_id` will perform at a given date. For that reason, we start by fixing the data-frame so that it is given in
the format $t|i_1|\cdots|i_N$, where $t$ is the time (with a daily granularity), $i_k$ contains the amount the $k$-th user_id spent, and $N$ is the total number of user_id.

In [12]:
# Merge purchases and users
merged_df = pd.merge(
    purchases,
    users
)[['user_id', 'purchased_at', 'value', 'orig_1']]
# Fix target categorical cols
merged_df['user_id'] = merged_df.user_id.fillna(-1).astype(int).astype(str)
merged_df['orig_1'] = merged_df.orig_1.fillna(-1).astype(int).astype(str)
merged_df['t'] = merged_df.purchased_at.dt.date

# Reduce number of user_ids by 85%
merged_df = merged_df[
    merged_df.user_id.isin(
        merged_df.user_id.drop_duplicates().sample(
            frac = 0.15,
            random_state = 42
        )
    )
]

# Create hierarchy,
# - First creating top
hierarchy = {
    'total': merged_df.orig_1.unique().tolist()
}
# - Then for each possible origin, getting possible unique user_ids
for orig_1 in merged_df.orig_1.unique():
    hierarchy[orig_1] = merged_df[merged_df.orig_1 == orig_1].user_id.unique().tolist()
    hierarchy[orig_1] = [orig_1 + '_' + user_id for user_id in hierarchy[orig_1]]

# Create Y in scikit-hts format,
# - first creating bottom time series:
Y_bottom = merged_df.groupby(
    ['t', 'orig_1', 'user_id']
).value.sum().reset_index().pivot(
    index='t',
    columns=['orig_1', 'user_id'],
    values='value'
)
Y_bottom.index = pd.to_datetime(Y_bottom.index)
Y_bottom = Y_bottom.resample('90d').sum()
Y_bottom

orig_1            30                            78        30        39  \
user_id         1690      2020      2023       945      2045       418   
t                                                                        
2017-08-12  0.472902  1.212912  0.951341  0.891576  0.654973  0.497468   
2017-11-10  0.000000  0.000000  0.000000  0.000000  0.000000  0.489382   
2018-02-08  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-05-09  0.000000  0.000000  0.000000  0.760555  0.000000  0.000000   
2018-08-07  0.468287  0.000000  0.000000  0.000000  0.895556  0.000000   
2018-11-05  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019-02-03  0.498888  0.000000  1.710110  0.000000  1.021350  0.387298   
2019-05-04  0.387298  0.000000  1.201136  0.000000  0.000000  0.501009   
2019-08-02  0.000000  0.000000  1.229762  0.000000  0.223607  0.000000   
2019-10-31  0.000000  0.000000  0.525030  0.000000  0.000000  0.000000   
2020-01-29  0.000000  0.000000  0.887782  0.000000  0.000000  0.000000   
2020-04-28  0.000000  0.000000  1.288297  0.000000  0.000000  0.000000   
2020-07-27  0.000000  0.000000  0.000000  0.000000  1.437685  1.396054   
2020-10-25  0.000000  0.000000  0.851857  0.000000  1.230985  0.000000   
2021-01-23  0.372678  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-04-23  0.000000  0.000000  0.449017  0.000000  0.776940  0.000000   
2021-07-22  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

orig_1            30                            78  ...        49        75  \
user_id         2068      2083       639      2123  ...     99881     30978   
t                                                   ...                       
2017-08-12  0.517082  0.633094  0.511386  1.296848  ...  0.000000  0.000000   
2017-11-10  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
2018-02-08  0.000000  0.000000  1.546782  0.000000  ...  0.000000  0.000000   
2018-05-09  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
2018-08-07  0.000000  0.378861  0.948787  0.000000  ...  0.000000  0.000000   
2018-11-05  0.000000  0.000000  0.895556  0.000000  ...  0.000000  0.000000   
2019-02-03  0.000000  0.000000  0.895556  0.000000  ...  0.000000  0.000000   
2019-05-04  0.000000  0.000000  0.447778  0.000000  ...  0.000000  0.000000   
2019-08-02  0.000000  0.223607  0.000000  0.000000  ...  0.000000  0.000000   
2019-10-31  0.000000  0.000000  0.387298  0.000000  ...  0.000000  0.000000   
2020-01-29  0.000000  0.000000  0.387298  0.000000  ...  0.000000  0.000000   
2020-04-28  0.000000  0.000000  0.000000  0.939496  ...  0.000000  0.000000   
2020-07-27  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
2020-10-25  0.000000  0.000000  0.431699  0.000000  ...  0.000000  0.000000   
2021-01-23  0.000000  0.396194  0.000000  0.000000  ...  0.000000  0.000000   
2021-04-23  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
2021-07-22  0.000000  0.000000  0.000000  0.000000  ...  0.421158  2.215647   

orig_1            19        30                                          \
user_id        99979     93382     99946     99950     99976     99983   
t                                                                        
2017-08-12  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2017-11-10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-02-08  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-05-09  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-08-07  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-11-05  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019-02-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019-05-04  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019-08-02  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2019-10-31  0.000000  0.000000  0.000000  0.000000

In [13]:
Y_bottom.columns = ["_".join(cols) for cols in Y_bottom.columns]
# - then middle - origin_1 - columns
Y_mid = merged_df.groupby(
    ['t', 'orig_1']
).value.sum().reset_index().pivot(
    index='t',
    columns=['orig_1'],
    values='value'
)
Y_mid.index = pd.to_datetime(Y_mid.index)
Y_mid = Y_mid.resample('90d').sum()
# finally total
Y_total = Y_mid.sum(axis=1).rename('total')
# and concatenating all of them
Y = pd.concat(
    (Y_bottom, Y_mid, Y_total), axis=1
)
Y

,30_1690,30_2020,30_2023,78_945,30_2045,39_418,30_2068,30_2083,30_639,78_2123,...,79,8,80,83,85,87,88,89,95,total
t,,,,,,,,,,,,,,,,,,,,,
2017-08-12,0.472902,1.212912,0.951341,0.891576,0.654973,0.497468,0.517082,0.633094,0.511386,1.296848,...,0.000000,0.000000,0.000000,0.417665,0.000000,0.000000,0.000000,0.000000,0.000000,19.564772
2017-11-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.489382,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.663450
2018-02-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.546782,0.000000,...,0.000000,0.000000,0.000000,1.272030,0.000000,0.000000,0.000000,0.000000,0.000000,58.929340
2018-05-09,0.000000,0.000000,0.000000,0.760555,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.304958,0.000000,0.000000,1.214059,0.000000,0.472261,0.000000,0.000000,0.000000,89.443992
2018-08-07,0.468287,0.000000,0.000000,0.000000,0.895556,0.000000,0.000000,0.378861,0.948787,0.000000,...,3.276220,0.000000,0.000000,0.447778,0.467640,0.000000,0.000000,0.000000,0.000000,115.585226
2018-11-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.895556,0.000000,...,1.158386,0.000000,0.000000,4.757434,0.000000,0.000000,0.000000,0.000000,0.000000,159.166736
2019-02-03,0.498888,0.000000,1.710110,0.000000,1.021350,0.387298,0.000000,0.000000,0.895556,0.000000,...,1.424082,0.000000,0.000000,2.218464,0.000000,0.399747,0.000000,0.000000,0.000000,179.026522
2019-05-04,0.387298,0.000000,1.201136,0.000000,0.000000,0.501009,0.000000,0.000000,0.447778,0.000000,...,10.488206,0.000000,2.084392,4.906952,1.201136,0.000000,0.000000,0.000000,0.000000,375.093596
2019-08-02,0.000000,0.000000,1.229762,0.000000,0.223607,0.000000,0.000000,0.223607,0.000000,0.000000,...,10.275672,0.000000,0.402141,4.165746,0.400379,0.000000,0.000000,0.000000,0.000000,596.642750


In [14]:
from hts import HTSRegressor

hts_regressor = HTSRegressor(
    model = 'holt_winters',
    revision_method = 'WLSS',
    n_jobs = 15,
    low_memory = True
)
hts_regressor.fit(df = Y.iloc[:-1], nodes = hierarchy)
predictions = hts_regressor.predict(steps_ahead=1)

Fitting models: 100%|██████████| 75/75 [00:06<00:00, 11.10it/s]


In [31]:
import numpy as np
absolute_errors = {
    'model' : list(),
    'naive' : list()
}
for col in Y_bottom.columns :
    absolute_errors['model'].append(np.abs(Y[col].iloc[-1] - predictions[col].iloc[-1]))
    absolute_errors['naive'].append(np.abs(Y[col].iloc[-1] - Y[col].iloc[-2]))


In [37]:
import hts.hierarchy, hts.functions
tree = hts.hierarchy.HierarchyTree.from_nodes(df = Y, nodes=hierarchy)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

In [55]:
forecasts = Y.iloc[[-2]]
import collections
pred_dict = collections.OrderedDict()

# Add predictions to dictionary is same order as summing matrix
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=forecasts[label].values, columns=['yhat'])

In [60]:
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})

In [50]:
import hts.functions
import statsmodels, collections, tqdm

forecasts = pd.DataFrame(columns=Y.columns)

# Make forecasts made outside of package. Could be any modeling technique.
for col in tqdm.tqdm(Y.columns):
    model = statsmodels.tsa.holtwinters.SimpleExpSmoothing(Y[col].values).fit()
    fcst = list(model.forecast(90))
    forecasts[col] = fcst

pred_dict = collections.OrderedDict()

 17%|█▋        | 1124/6765 [00:05<00:27, 203.05it/s]


KeyboardInterrupt: 

In [ ]:
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

# Add predictions to dictionary is same order as summing matrix
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=forecasts[label].values, columns=['yhat'])

In [ ]:
pred_dict

In [ ]:
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='WLSV', mse={})

# Put reconciled forecasts in nice DataFrame form
revised_forecasts = pd.DataFrame(
    data=revised[0:,0:],
    index=forecasts.index,
    columns=sum_mat_labels
)

In [ ]:
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='WLSV', mse={})

In [13]:
hts_regressor.predict(steps_ahead = 10)

Fitting models:   0%|          | 0/50 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/Library/F

KeyboardInterrupt: 